<a href="https://colab.research.google.com/github/dtabuena/CrispyMice/blob/main/New_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)
'''Get Repositories'''


'Get Repositories'

In [1]:

try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib

to_import = [
          'ABF_Quality_Control.ipynb',
          'Basic_Ephys.ipynb',
          'Firing_Rate_Gain.ipynb',
          'Simple_ABF_tools.ipynb',
          'IV_analyzer.ipynb',
          'Vm_analyzer.ipynb',
          'membrane_analyzer.ipynb',
          'analyze_rheobase.ipynb',
          'fun_math.ipynb',
          'importing_abfs_from_dropbox.ipynb',
          'input_resistance_analyzer.ipynb',
          'latencey_analyzer.ipynb',
          'QC_recoding_dataframe.ipynb'
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


try: shutil.rmtree('/content/CrispyMice')
except: None
!git clone https://github.com/dtabuena/CrispyMice
to_import = ['CrispyPipelineFuncs.ipynb']
for i in to_import:
    f = '/content/CrispyMice/' + i
    %run $f

Cloning into 'EphysLib'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 244 (delta 100), reused 29 (delta 29), pack-reused 94
Receiving objects: 100% (244/244), 204.30 KiB | 5.24 MiB/s, done.
Resolving deltas: 100% (144/144), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.3 MB/s 
Cloning into 'CrispyMice'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 122 (delta 47), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (122/122), 3.47 MiB | 6.50 MiB/s, done.
Resolving deltas: 100% (47/47), done.
